In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "How many times do you want to apply the Perceptron and MLP network ? 100\n",
      "('The Perceptron average accuracy of:', 100, 'experiments on raw data is:', 0.7612743823146944)\n",
      "('The Perceptron average accuracy of:', 100, 'experiments on preprocessed data is:', 0.763107932379714)\n",
      "\n",
      "('The MLP average accuracy of:', 100, 'experiments on raw data is:', 0.5035546875)\n",
      "('The MLP average accuracy of:', 100, 'experiments on preprocessed data is:', 0.49677042801556426)\n"
     ]
    }
   ],
   "source": [
    "#1)Your program will first read the above file for the Heart Disease patients.  \n",
    "import numpy as np # Use for numpy array\n",
    "import csv # for file open\n",
    "from Perceptron import PCN # Import my PCN implmentation \n",
    "from Helper_Function import sigmoid_act, sigmoid_derivative # Import our activation and  sigmoid functions \n",
    "\n",
    "\n",
    "with open(\"heart2.csv\",'r') as file_h:\n",
    "        csvreader = csv.reader(file_h)\n",
    "        #Delcare our data set as a list\n",
    "        heart_data = []\n",
    "        for row in csvreader: # for each row in the csvreader\n",
    "            try: # try append val floats for each val in the row on csv reader\n",
    "                  heart_data.append([float(val) for val in row])\n",
    "            except ValueError: # will will continue if there is non-numeric values such as age, gender...etc\n",
    "                  continue\n",
    "        #Using numpy we can use this into a array for numpy\n",
    "        heart_data = np.array(heart_data) # assigned heart_data into numpy array        \n",
    "        #Step 2 \n",
    "\n",
    "iterations = int(input(\"How many times do you want to apply the Perceptron and MLP network ? \"))\n",
    "count = 0\n",
    "#Iniatlize our data sets for raw and preprocessed for PCN\n",
    "preprocesses_data_pcn = []\n",
    "raw_data_pcn = []\n",
    "\n",
    "##Iniatlize our data sets for raw and preprocessed for MLP\n",
    "preprocesses_data_MLP = []\n",
    "raw_data_MLP = []\n",
    "\n",
    "\n",
    "patience = 10  # Adjust this value as needed\n",
    "best_pre_accur_mlp = 0\n",
    "epoch_best = 0 # for epoch on early stopping\n",
    "\n",
    "#Assign our lengths and sizes for MLP\n",
    "size_output = 1 #define output level\n",
    "size_hidden_layer = 100 #hidden layer size 100\n",
    "size_input = len(heart_data[0]) -1 # size of input layer subtact 1 from number of features in heart\n",
    "\n",
    "#For loop assigned to our iteartions\n",
    "for i in range (iterations): \n",
    "      \n",
    "      #Random shuffle compute\n",
    "      np.random.shuffle(heart_data)\n",
    "\n",
    "      #Separate a quarter of the data for training and the other quarter for testing,\n",
    "      # By assigning our boudnaries for testing and trainng data\n",
    "      training_size = int (0.25 *len(heart_data)) # Calculatye size of train set quarter\n",
    "      training_data = heart_data[:training_size] #Create training set\n",
    "      testing_data = heart_data[training_size:] #CReate testing set\n",
    "\n",
    "      #PCN Initialization\n",
    "      pcn = PCN (operations = len(training_data[0])-1) #Initial instance of PCN\n",
    "      x_train = training_data[:,:-1] # assign our x on trainng\n",
    "      y_train = training_data[:,-1] # assign our y on training\n",
    "      pcn.train_compute (x_train,y_train) # train our x and y\n",
    "\n",
    "      #Apply PCN for preprocessing:\n",
    "      #subtact min value by scale of range and sub min value for preprocvessing on training data\n",
    "      train_pre_data = (training_data - np.min(training_data, axis = 0) / np.max(training_data, axis = 0) - np.min(training_data, axis = 0)) # Process pre-processed data\n",
    "      #Similar for preprocess but for testing \n",
    "      test_pro_data = (testing_data - np.min(training_data, axis = 0) / np.max(training_data, axis = 0) - np.min(training_data, axis = 0)) # Process processed test data\n",
    "\n",
    "      #Grab input feature and label from target from pree processed data training\n",
    "      pre_x_train_pcn = train_pre_data [:,:-1]\n",
    "      pre_y_train_pcn = train_pre_data [:, -1]\n",
    "\n",
    "      #Initalize PCN for preprocessed data\n",
    "      pcn_pre = PCN(operations = len(train_pre_data[0]) - 1 )\n",
    "      pcn_pre.train_compute(pre_x_train_pcn,pre_y_train_pcn) #Train Preprocess data\n",
    "      test_pcn_x_pre = test_pro_data[:,:-1] #Grab all input feature from test_pcn_y_pre\n",
    "      test_pcn_y_pre = test_pro_data[:,-1] ##Grab all input feature from test_pcn_y_pre\n",
    "\n",
    "      #Now we calcualte the accuracies of our PCN\n",
    "      pcn_acc_raw = np.mean(pcn.Predict(testing_data[:,:-1]) == testing_data[:,-1])\n",
    "      pcn_acc_preprocessed = np.mean(pcn_pre.Predict(test_pcn_x_pre) == test_pcn_y_pre) \n",
    "\n",
    "      #Iniatalize our MLP for data and implementation\n",
    "      mlp_size_train = int(0.50 * len(heart_data)) #Calculate size of MLP train whihch is half\n",
    "      mlp_size_valid = int(0.25 * len(heart_data)) #Calculate size of MLP train whihch is quarter\n",
    "      training_data_mlp = heart_data [:mlp_size_train] #Creqate training data from the rows of mlp_size_train\n",
    "      validation_data_mlp = heart_data [mlp_size_train:mlp_size_train +mlp_size_valid] #Create validatio data mlp from rows mlp_size_train to mlp_size_valid \n",
    "      testing_data_mlp =  heart_data [mlp_size_train + mlp_size_valid:] #Create testing data for rows mlp_size_train and mlp_size_valid\n",
    "\n",
    "      # MLP apply normilaization:\n",
    "      val_max = np.max(training_data_mlp, axis = 0)  #Calculate the max of training with respect to our axis\n",
    "      val_min = np.min(training_data_mlp, axis = 0) #Calcuale the min of training with respetc to our axis\n",
    "      \n",
    "      for index_feature in range(size_input): # Used for overload issues wince we can compute this part seperately instead in one go\n",
    "            min_val = val_min[index_feature]\n",
    "            max_val = val_max[index_feature]\n",
    "            training_data_mlp[:,index_feature] = (training_data_mlp[:,index_feature] - min_val) / (max_val-min_val)  #Normialize our data by subtact train - min / max -min\n",
    "            validation_data_mlp [:,index_feature] = (validation_data_mlp[:,index_feature] - min_val) / (max_val-min_val)  #Normalize our data by sub valid - min  max - min\n",
    "            testing_data_mlp [:,index_feature] = (testing_data_mlp[:,index_feature] - min_val) / (max_val-min_val) #Normailize our test - val / max - min\n",
    "\n",
    "            # training_data_mlp = (training_data_mlp - val_min) / (val_max - val_min) #Normialize our data by subtact train - min / max -min\n",
    "            # validation_data_mlp = (validation_data_mlp - val_min) / (val_max - val_min) #Normalize our data by sub valid - min  max - min\n",
    "            # testing_data_mlp = (testing_data_mlp - val_min) / (val_max - val_min) #Normailize our test - val / max - min\n",
    "\n",
    "      #Extract our inputs from x train and y train frrom our training mlp data \n",
    "      x_train_mlp, y_train_mlp = training_data_mlp [:,:-1] , training_data_mlp [:,- 1]\n",
    "      #Extract our inputs of x an y validation from our validation set\n",
    "      x_val_mlp, y_val_mlp = validation_data_mlp[:,:-1], validation_data_mlp[:,-1]\n",
    "      #Extract our x and y test from our testing set\n",
    "      x_test_mlp, test_y_mlp = testing_data_mlp[:,:-1], testing_data_mlp[:,-1]\n",
    "\n",
    "      #Assign our weights and bias for MLP hidden inputs and outpust\n",
    "      input_hidden_weights = 2 * np.random.rand(size_input,size_hidden_layer) -1\n",
    "      output_hidden_weights = 2 * np.random.rand(size_hidden_layer,size_output) -1\n",
    "      #Initalzie random weight for input and hidden layer creates matrix of these weights\n",
    "\n",
    "      \n",
    "\n",
    "      #Multi-Level Perceptron (MLP) for a fixed number of iterations (say, several hundred or thousand \n",
    "      for epoch in range(1000):\n",
    "        \n",
    "        \n",
    "\n",
    "        #for our layers (FowardPropagation)\n",
    "        input_hidden = np.dot(x_train_mlp,input_hidden_weights) #Calcuate the weighted sum of hidden layer\n",
    "        output_hidden = sigmoid_act(input_hidden) # Calculate sigmoid activation function for hidden layer sum output\n",
    "        output_input_layer = np.dot(output_hidden,output_hidden_weights) #Calculate weighted sum of inputs of output layer\n",
    "        output_output_layer = sigmoid_act(output_input_layer) #Apply sigmoid on output layers\n",
    "\n",
    "        # (Back Propagation) Note-> d output variable and rest of d are denoted as derivative\n",
    "        error_back = y_train_mlp.reshape(-1,1) - output_output_layer #perfom error calculaytion of target table\n",
    "        d_output = error_back * sigmoid_derivative(output_output_layer) #Calculate derivative of error by applyoh sigmoid_derivative\n",
    "        error_back_hidden = d_output.dot (output_hidden_weights.T) #Calculaye backpropagtion of error on the output layer\n",
    "        d_hidden = error_back_hidden * sigmoid_derivative(output_hidden) #Calculate derivative of hidden layer using sigmoid derivative\n",
    "\n",
    "        #Adjust our bias and weights of MLP Note-> T (is our transpose operation for our martix)\n",
    "        output_hidden_weights += output_hidden.T.dot(d_output) #Adjust weights by adding out hidden layer output \n",
    "        input_hidden_weights += x_train_mlp.T.dot(d_hidden) #Adjust weights for input layer \n",
    "\n",
    "      #Perfom calculation of MLP raw data\n",
    "        mlp_accuracy_raw = np.mean(np.round(output_output_layer) == y_train_mlp) #Perform our eound on MLP output againts  our y_train_mlp for accuracy \n",
    "        #Perform calculation on preprocessed MLP data\n",
    "        input_hidden = np.dot(x_test_mlp,input_hidden_weights) # Calculate wighted sum  of input_hidden _weights\n",
    "        output_hidden = sigmoid_act(input_hidden) #Calculate our sigmoid with input_hidden\n",
    "        output_input_layer = np.dot(output_hidden,output_hidden_weights) #Calculate wighted sum of hidden layers\n",
    "        output_output_layer = sigmoid_act(output_input_layer) # Pass again to activation functpon used for our final calculation of accuracy for preprocessed\n",
    "        mlp_accuracy_pre = np.mean(np.round(output_output_layer) == test_y_mlp)\n",
    "\n",
    "        #Early Stopping applied.\n",
    "        if mlp_accuracy_pre > best_pre_accur_mlp : # check is current mlp accuracy is our best one\n",
    "            best_pre_accur_mlp = mlp_accuracy_pre # if it is assign both accordinly\n",
    "            best_epoch = epoch # upate best_epoch to the current\n",
    "\n",
    "        if  epoch - best_epoch > patience: # else no improvemnnt then we apply early stopping\n",
    "            break\n",
    "\n",
    "\n",
    "      #Append our calc to list\n",
    "      raw_data_pcn.append(pcn_acc_raw)\n",
    "      preprocesses_data_pcn.append(pcn_acc_preprocessed)\n",
    "      preprocesses_data_MLP.append(mlp_accuracy_pre)\n",
    "      raw_data_MLP.append(mlp_accuracy_raw)\n",
    "\n",
    "      # count += 1\n",
    "      # print (\"Iteartions:\", count)\n",
    "\n",
    "\n",
    "#Finally we can calculate our averae acuracies MLP and PCN\n",
    "a_pcn_raw = np.mean(raw_data_pcn)\n",
    "a_pcn_pre = np.mean(preprocesses_data_pcn)\n",
    "a_mlp_raw = np.mean(raw_data_MLP)\n",
    "a_mlp_pre = np.mean(preprocesses_data_MLP)\n",
    "\n",
    "#Print out average accuracies\n",
    "print(\"The Perceptron average accuracy of:\", iterations, \"experiments on raw data is:\", a_pcn_raw)\n",
    "print(\"The Perceptron average accuracy of:\", iterations, \"experiments on preprocessed data is:\", a_pcn_pre)\n",
    "print(\"\")\n",
    "print(\"The MLP average accuracy of:\", iterations, \"experiments on raw data is:\", a_mlp_raw)\n",
    "print(\"The MLP average accuracy of:\", iterations, \"experiments on preprocessed data is:\", a_mlp_pre)\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "     \n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "\n",
    "      \n",
    "      "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 2",
   "language": "python",
   "name": "python2"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 2
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython2",
   "version": "2.7.13"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
